# 6.8620 Final Project: Sarcasm + Emotion

* [proposal](https://www.overleaf.com/project/662876554d9d1a627cdd74d3)
* pset refs:
    * *[ps3](https://colab.research.google.com/drive/1yaPzyM9dKCRH1huGqNW1gtyTPet_P4a5?pli=1&usp=drive_fs#scrollTo=gXCHTYCp2FGJ): acoustic models (data processing)
    * [ps4](https://colab.research.google.com/drive/1BKiIERrEerfkm8Z0bPZcLRLXMjvytmBo?pli=1&usp=drive_fs): forced alignments, HMM, HMM-GMM, neural alignment (terminal data)
    * [ps5](https://colab.research.google.com/drive/1jlkOQdIkWP3LmaSsdE9VMtImU8QA_dng?pli=1&usp=drive_fs): n-gram LMs (terminal data)
    * [ps6](https://colab.research.google.com/drive/17uz4Ryfwm9F3s4Jhv1yYsYeQHQs6h408?pli=1&usp=drive_fs): neural LMs (nltk brown)
    * *[ps7](https://colab.research.google.com/drive/1Dx5392Ph_lE3J9bvWXhQiabS11YtfK8W?pli=1&usp=drive_fs): end-to-end ASR
    * **[ps8](https://colab.research.google.com/drive/1235jJYQgfHRuX_TbU7GMTpzlJIKYrDDc?pli=1&usp=drive_fs): wav2vec, xlsr (Wav2vecFeatureExtractor section)

# Initialization

In [1]:
!pip install -qU transformers datasets evaluate accelerate peft huggingface_hub peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pickle
import numpy as np
import json
import librosa
from IPython.display import Audio as AudioDisplay
import pandas as pd
import soundfile as sf
import random
import time
import tqdm
from sklearn.metrics import accuracy_score

from transformers import HubertModel, HubertConfig, PretrainedConfig, PreTrainedModel, TrainingArguments, Trainer
from transformers import AutoModel, AutoFeatureExtractor, AutoModelForAudioClassification, AutoProcessor
from datasets import load_dataset, Audio, Dataset
import evaluate
from peft import get_peft_model, LoraConfig, TaskType

In [3]:
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def mount_gdrive():
    from google.colab import drive
    drive.mount('/content/gdrive')

In [4]:
SEED = 14
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Data

## General

In [5]:
# https://huggingface.co/docs/transformers/en/model_doc/wav2vec2#transformers.Wav2Vec2Processor
hubert_processor = AutoProcessor.from_pretrained('facebook/hubert-large-ls960-ft') # is using a processor different from the model ok?

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [6]:
# using processor: https://huggingface.co/docs/transformers/en/model_doc/wav2vec2#transformers.Wav2Vec2FeatureExtractor
def preprocess_dataset(batch):
    output = hubert_processor(
        audio=[audio['array'] for audio in batch['audio']],
        sampling_rate=16_000,
        padding='longest',
        return_tensors='pt',
    )
    return output

In [7]:
def get_dataset_splits(ds):
    train_valtest = ds.train_test_split(test_size=0.2, seed=SEED)
    val_test = train_valtest['test'].train_test_split(test_size=0.5, seed=SEED)

    train_set = train_valtest['train']
    val_set = val_test['train']
    test_set = val_test['test']

    return train_set, val_set, test_set

In [8]:
def onehot_e(batch):
    label = torch.zeros(8)
    label[batch['labels']-1] = 1
    batch['one_hot'] = label.to(torch.int64)
    return batch

## MUStARD: sarcasm detection

([paper](https://aclanthology.org/P19-1455/), [github](https://github.com/soujanyaporia/MUStARD))

In [9]:
!gdown 1--U_CMuzrXZ7t01sFgeh0gKp-V8_D6pz
!unzip sarcasm_ds.zip

Downloading...
From (original): https://drive.google.com/uc?id=1--U_CMuzrXZ7t01sFgeh0gKp-V8_D6pz
From (redirected): https://drive.google.com/uc?id=1--U_CMuzrXZ7t01sFgeh0gKp-V8_D6pz&confirm=t&uuid=05e2556e-6594-4022-825a-3cda346ce21e
To: /content/sarcasm_ds.zip
100% 218M/218M [00:03<00:00, 60.3MB/s]
Archive:  sarcasm_ds.zip
   creating: sarcasm_ds/
  inflating: sarcasm_ds/data-00000-of-00004.arrow  
  inflating: sarcasm_ds/data-00001-of-00004.arrow  
  inflating: sarcasm_ds/state.json   
  inflating: sarcasm_ds/data-00003-of-00004.arrow  
  inflating: sarcasm_ds/data-00002-of-00004.arrow  
  inflating: sarcasm_ds/dataset_info.json  


In [10]:
sarcasm_ds = Dataset.load_from_disk('sarcasm_ds')
print(sarcasm_ds.features)
print(sarcasm_ds)

{'identifier': Value(dtype='string', id=None), 'labels': Value(dtype='int64', id=None), 'input_values': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)}
Dataset({
    features: ['identifier', 'labels', 'input_values', 'attention_mask'],
    num_rows: 690
})


In [11]:
sarcasm_train_set, sarcasm_val_set, sarcasm_test_set = get_dataset_splits(sarcasm_ds)
print(sarcasm_train_set.features)
print(sarcasm_train_set)

sarcasm_train_loader = DataLoader(sarcasm_train_set, batch_size=2)
sarcasm_val_loader = DataLoader(sarcasm_val_set, batch_size=2)
sarcasm_test_loader = DataLoader(sarcasm_test_set, batch_size=2)

{'identifier': Value(dtype='string', id=None), 'labels': Value(dtype='int64', id=None), 'input_values': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)}
Dataset({
    features: ['identifier', 'labels', 'input_values', 'attention_mask'],
    num_rows: 552
})


In [12]:
print(sarcasm_train_set[0])

{'identifier': '2_111', 'labels': tensor(0), 'input_values': tensor([-0.1981, -0.6425, -0.3329,  ...,  0.0000,  0.0000,  0.0000]), 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0])}


## RAVDESS: emotion detection

([website](https://zenodo.org/records/1188976#.YFZuJ0j7SL8))

To keep dataset sizes comparable, take 100 samples of each of the 6 emotions in RAVDESS to create a sampled dataset of ~600 examples, similar size to MUStARD

In [13]:
!gdown 1UfcwmyRjTGRP1g4NQa92Uzs9z0PH-hil
!unzip emotion_ds.zip

Downloading...
From (original): https://drive.google.com/uc?id=1UfcwmyRjTGRP1g4NQa92Uzs9z0PH-hil
From (redirected): https://drive.google.com/uc?id=1UfcwmyRjTGRP1g4NQa92Uzs9z0PH-hil&confirm=t&uuid=ffaefa47-2f88-4c4c-8083-183f87e74203
To: /content/emotion_ds.zip
100% 151M/151M [00:01<00:00, 98.2MB/s]
Archive:  emotion_ds.zip
   creating: emotion_dataset/
   creating: emotion_ds/
  inflating: emotion_ds/state.json   
  inflating: emotion_ds/data-00000-of-00001.arrow  
  inflating: emotion_ds/dataset_info.json  


In [14]:
emotion_ds = Dataset.load_from_disk('emotion_ds')
print(emotion_ds.features)
print(emotion_ds)

{'labels': Value(dtype='int64', id=None), 'intensity': Value(dtype='int64', id=None), 'statement': Value(dtype='int64', id=None), 'actor': Value(dtype='int64', id=None), 'input_values': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)}
Dataset({
    features: ['labels', 'intensity', 'statement', 'actor', 'input_values', 'attention_mask'],
    num_rows: 720
})


In [15]:
emotion_train_set, emotion_val_set, emotion_test_set = get_dataset_splits(emotion_ds)
print(emotion_train_set.features)
print(emotion_train_set)

emotion_train_loader = DataLoader(emotion_train_set, batch_size=2)
emotion_val_loader = DataLoader(emotion_val_set, batch_size=2)
emotion_test_loader = DataLoader(emotion_test_set, batch_size=2)

{'labels': Value(dtype='int64', id=None), 'intensity': Value(dtype='int64', id=None), 'statement': Value(dtype='int64', id=None), 'actor': Value(dtype='int64', id=None), 'input_values': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)}
Dataset({
    features: ['labels', 'intensity', 'statement', 'actor', 'input_values', 'attention_mask'],
    num_rows: 576
})


In [16]:
print(emotion_train_set[0])

{'labels': tensor(3), 'intensity': tensor(2), 'statement': tensor(1), 'actor': tensor(23), 'input_values': tensor([0.0004, 0.0016, 0.0004,  ..., 0.0000, 0.0000, 0.0000]), 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0])}


In [17]:
emotions = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised'] # aligned with ravdess' definition
emotion_id2label = {}
emotion_label2id = {}
for i in range(8): # each id is one less than what is defined by ravdess (eg. neutral in ravdess = 1, neutral id here = 0)
    emotion_id2label[i] = emotions[i-1]
    emotion_label2id[emotions[i-1]] = i
emotion_id2label

{0: 'surprised',
 1: 'neutral',
 2: 'calm',
 3: 'happy',
 4: 'sad',
 5: 'angry',
 6: 'fearful',
 7: 'disgust'}

# Models

## HuBERT

([paper](https://arxiv.org/abs/2106.07447), [github](https://github.com/facebookresearch/fairseq/tree/main/examples/hubert), [documentation](https://huggingface.co/docs/transformers/en/model_doc/hubert#transformers.HubertModel), [model card](https://huggingface.co/facebook/hubert-base-ls960))

## General

In [18]:
hubert_model_name = 'facebook/hubert-base-ls960'
hubert_model = HubertModel.from_pretrained(hubert_model_name) # 94M params

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

In [19]:
# lora_config = LoraConfig(
#             task_type=TaskType.FEATURE_EXTRACTION, #r=8, lora_alpha=32, lora_dropout=0.1
# )
# peft_hubert_model = get_peft_model(hubert_model, lora_config)
# print(count_params(peft_hubert_model))

In [20]:
class HubertClassifier(nn.Module):
    def __init__(
            self,
            hubert_model_name=hubert_model_name, # 'facebook/hubert-base-ls960'
            num_classes=2,
            transplant_hubert=None,
            transplant_classifier=None,
    ):
        super().__init__()
        if transplant_hubert is None:
            self.hubert = HubertModel.from_pretrained(hubert_model_name)
        else:
            print('transplanted hubert!')
            self.hubert = transplant_hubert
        # TODO: PEFT?

        if transplant_classifier is None:
            self.classifier = nn.Sequential(
                nn.Linear(768, 32),
                nn.ReLU(),
                nn.Dropout(0.1),
                nn.Linear(32, num_classes)
            )
        else:
            print('transplanted classifier!')
            self.classifier = transplant_classifier

        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input_values=None, attention_mask=None, labels=None):
        hubert_output = self.hubert(input_values=input_values, attention_mask=attention_mask)
        pooler_output = torch.mean(hubert_output.last_hidden_state, dim=1) # replace with PEFT?
        logits = self.classifier(pooler_output)
        out = {
            'logits': logits,
            'pooler_output': pooler_output,
        }

        if labels is not None:
            loss = self.loss_fn(logits, labels)
            out['loss'] = loss

        return out

In [21]:
# custom huggingface config/model definition
# https://huggingface.co/docs/transformers/en/custom_models
class HubertClassifierConfig(PretrainedConfig):
    def __init__(
            self,
            hubert_model_name = hubert_model_name,
            num_classes = 2,
            **kwargs,
    ):
        self.hubert_model_name = hubert_model_name
        self.num_classes = num_classes
        super().__init__(**kwargs)

class HubertClassifierModel(PreTrainedModel):
    config_class = HubertClassifierConfig

    def __init__(self, config):
        super().__init__(config)
        self.model = HubertClassifier(
            hubert_model_name=config.hubert_model_name,
            num_classes=config.num_classes
        )

    def forward(self, input_values=None, attention_mask=None, labels=None):
        return self.model.forward(input_values=input_values, attention_mask=attention_mask, labels=labels)

In [27]:
def train(hubert_classifier, train_loader, val_loader, num_epochs=2):
    train_losses, val_losses = [], []

    optimizer = torch.optim.Adam(hubert_classifier.parameters(), lr=5e-3)
    hubert_classifier = hubert_classifier.to(DEVICE)
    start_time = time.time()

    for epoch in range(num_epochs):
        hubert_classifier.train()
        epoch_train_loss = 0
        for ix, batch in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
            input_values = batch['input_values'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)

            out = hubert_classifier(input_values=input_values, attention_mask=attention_mask, labels=labels)
            print(f'{out["logits"]=}')
            print(f'{batch["labels"]=}')

            loss = out['loss']
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            epoch_train_loss += loss.item()
        epoch_train_loss /= len(train_loader)
        train_losses.append(epoch_train_loss)

        hubert_classifier.eval()
        epoch_val_loss = 0
        for ix, batch in tqdm.tqdm(enumerate(val_loader), total=len(val_loader)):
            input_values = batch['input_values'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)

            out = hubert_classifier(input_values=input_values, attention_mask=attention_mask, labels=labels)
            loss = out['loss']
            epoch_val_loss += loss.item()
        epoch_val_loss /= len(val_loader)
        val_losses.append(epoch_val_loss)


        print(f'Epoch {epoch+1}: Loss (train/val): {epoch_train_loss}/{epoch_val_loss}')
        print()

    print(f'Total training time elapsed: {round(time.time() - start_time, 2)}')

    return train_losses, val_losses

In [23]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(f'{predictions=}')
    print(f'{predictions.shape=}')
    predictions = np.argmax(predictions, axis=1)
    return accuracy_score(labels, predictions)

In [24]:
h_config = HubertClassifierConfig(num_classes=2)
h_model = HubertClassifierModel(h_config)
print(h_model)

Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

HubertClassifierModel(
  (model): HubertClassifier(
    (hubert): HubertModel(
      (feature_extractor): HubertFeatureEncoder(
        (conv_layers): ModuleList(
          (0): HubertGroupNormConvLayer(
            (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
            (activation): GELUActivation()
            (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
          )
          (1-4): 4 x HubertNoLayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
            (activation): GELUActivation()
          )
          (5-6): 2 x HubertNoLayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
            (activation): GELUActivation()
          )
        )
      )
      (feature_projection): HubertFeatureProjection(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (projection): Linear(in_features=512, out_features=768, bias=True)
  

## Sarcasm Hubert

In [29]:
# base model testing
sarcasm_hubert = HubertClassifier(num_classes=2)

train_losses, val_losses = train(sarcasm_hubert, sarcasm_train_loader, sarcasm_val_loader, num_epochs=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model ch

out["logits"]=tensor([[0.2056, 0.0096],
        [0.2051, 0.0150]], device='cuda:0', grad_fn=<AddmmBackward0>)
batch["labels"]=tensor([0, 1])


  0%|          | 1/276 [00:00<01:35,  2.87it/s]

out["logits"]=tensor([[1.0095, 0.0211],
        [1.0253, 0.0140]], device='cuda:0', grad_fn=<AddmmBackward0>)
batch["labels"]=tensor([0, 1])


  1%|          | 3/276 [00:00<01:23,  3.25it/s]

...

out["logits"]=tensor([[ 0.1358, -0.0417],
        [ 0.1358, -0.0417]], device='cuda:0', grad_fn=<AddmmBackward0>)
batch["labels"]=tensor([1, 0])


100%|█████████▉| 275/276 [01:26<00:00,  3.16it/s]

out["logits"]=tensor([[ 0.1378, -0.0437],
        [ 0.1378, -0.0437]], device='cuda:0', grad_fn=<AddmmBackward0>)
batch["labels"]=tensor([0, 1])


100%|██████████| 35/35 [00:03<00:00,  9.80it/s]

Epoch 1: Loss (train/val): 0.7100810260930355/0.6921319995607649

Total training time elapsed: 90.8


In [ ]:
# HF model
sarcasm_hubert_classifier_config = HubertClassifierConfig(num_classes=2)
sarcasm_hubert_classifier_model = HubertClassifierModel(sarcasm_hubert_classifier_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model ch

In [ ]:
# HF trainer ref: https://huggingface.co/docs/transformers/main/en/trainer
# Trainer/TrainingArguments docs: https://huggingface.co/docs/transformers/main/en/main_classes/trainer
sarcasm_hubert_classifier_training_args = TrainingArguments(
    output_dir='sarcasm_hubert_classifier_model',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=5e-5,
    weight_decay=1e-2,
    num_train_epochs=10,
    # logging_steps=100,
    lr_scheduler_type='linear',
    evaluation_strategy='epoch', # eval_strategy?
    save_strategy='epoch',
    logging_strategy='epoch',
    # load_best_model_at_end=True,
)

sarcasm_hubert_classifier_trainer = Trainer(
    model=sarcasm_hubert_classifier_model,
    args=sarcasm_hubert_classifier_training_args,
    train_dataset=sarcasm_train_set,
    eval_dataset=sarcasm_val_set,
    # optimizers=( # need to specify optimizers explicity?
    #     torch.optim.AdamW,
    #     torch.optim.lr_scheduler.LinearLR
    # ),
)

In [ ]:
sarcasm_hubert_classifier_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss
1,0.701800,0.693337
2,0.700600,0.690571
3,0.700000,0.702648
4,0.693600,0.698065
5,0.696500,0.696815
6,0.692700,0.695547
7,0.695700,0.695526
8,0.693600,0.695361
9,0.694900,0.695256
10,0.691100,0.695565


TrainOutput(global_step=2760, training_loss=0.6960503453793733, metrics={'train_runtime': 1931.3202, 'train_samples_per_second': 2.858, 'train_steps_per_second': 1.429, 'total_flos': 0.0, 'train_loss': 0.6960503453793733, 'epoch': 10.0})

In [ ]:
mount_gdrive()
%cd /content/sarcasm_hubert_classifier_model
!zip -r checkpoint-1104.zip checkpoint-1104
!mv checkpoint-1104.zip /content/gdrive/MyDrive/2023-24/24spring/6.8620/project/models/sarcasm_hubert
%cd /content

/content/sarcasm_hubert_classifier_model
updating: checkpoint-1104/ (stored 0%)
updating: checkpoint-1104/optimizer.pt (deflated 7%)
updating: checkpoint-1104/training_args.bin (deflated 51%)
updating: checkpoint-1104/model.safetensors (deflated 7%)
updating: checkpoint-1104/config.json (deflated 24%)
updating: checkpoint-1104/trainer_state.json (deflated 71%)
updating: checkpoint-1104/rng_state.pth (deflated 25%)
updating: checkpoint-1104/scheduler.pt (deflated 55%)
/content


In [ ]:
!rm -rf sarcasm_hubert_classifier_model

## Emotion Hubert

In [60]:
# base model testing
emotion_hubert = HubertClassifier(num_classes=8)

train_losses, val_losses = train(emotion_hubert, emotion_train_loader, emotion_val_loader, num_epochs=4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model ch

input_values=tensor([[ 3.9280e-04,  1.5681e-03,  3.6434e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-5.0203e-06, -5.0203e-06, -5.0203e-06,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]], device='cuda:0')
labels=tensor([3, 0], device='cuda:0')
input_values=tensor([[ 1.2375e-04,  1.2375e-04,  1.2375e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.3493e-05, -1.3493e-05, -1.3493e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]], device='cuda:0')
labels=tensor([2, 6], device='cuda:0')


  1%|          | 3/288 [00:00<00:35,  7.95it/s]

input_values=tensor([[-1.7147e-03, -4.5738e-03, -3.9026e-03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 8.2095e-04, -7.1211e-04, -9.2799e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]], device='cuda:0')
labels=tensor([4, 1], device='cuda:0')
input_values=tensor([[ 2.5171e-04,  1.3527e-04, -5.7747e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 1.3138e-06,  1.7473e-06,  1.4404e-06,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]], device='cuda:0')
labels=tensor([2, 4], device='cuda:0')
input_values=tensor([[ 1.0400e-03, -2.4044e-03, -1.4948e-02,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 7.3416e-06,  7.3416e-06,  7.3416e-06,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]], device='cuda:0')
labels=tensor([4, 4], device='cuda:0')


  2%|▏         | 7/288 [00:00<00:27, 10.07it/s]

...

input_values=tensor([[ 0.0004,  0.0004,  0.0004,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0002, -0.0002, -0.0002,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')
labels=tensor([1, 7], device='cuda:0')
input_values=tensor([[-1.4482e-05, -1.4482e-05, -1.4482e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 2.5338e-03,  3.6020e-03,  3.7799e-03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]], device='cuda:0')
labels=tensor([7, 3], device='cuda:0')
input_values=tensor([[-1.1543e-05, -1.1072e-05, -1.2878e-05,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-1.7315e-02, -1.7450e-02, -3.8526e-03,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]], device='cuda:0')
labels=tensor([4, 1], device='cuda:0')


  7%|▋         | 21/288 [00:01<00:23, 11.32it/s]

input_values=tensor([[-0.0049, -0.0063, -0.0042,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0005, -0.0053, -0.0019,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')
labels=tensor([4, 4], device='cuda:0')
input_values=tensor([[-8.9162e-06, -8.9162e-06, -8.9162e-06,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 5.2640e-04,  5.2640e-04,  5.2640e-04,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]], device='cuda:0')
labels=tensor([2, 7], device='cuda:0')
input_values=tensor([[-0.0006, -0.0020, -0.0033,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0009, -0.0076, -0.0162,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')
labels=tensor([2, 3], device='cuda:0')


  8%|▊         | 23/288 [00:02<00:25, 10.57it/s]


KeyboardInterrupt: 

In [ ]:
# HF model
emotion_hubert_classifier_config = HubertClassifierConfig(num_classes=8)
emotion_hubert_classifier_model = HubertClassifierModel(emotion_hubert_classifier_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model ch

In [ ]:
# HF trainer ref: https://huggingface.co/docs/transformers/main/en/trainer
# Trainer/TrainingArguments docs: https://huggingface.co/docs/transformers/main/en/main_classes/trainer
emotion_hubert_classifier_training_args = TrainingArguments(
    output_dir='emotion_hubert_classifier_model',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=5e-5,
    weight_decay=1e-2,
    num_train_epochs=20,
    # logging_steps=100,
    lr_scheduler_type='linear',
    evaluation_strategy='epoch', # eval_strategy?
    save_strategy='epoch',
    logging_strategy='epoch',
    # load_best_model_at_end=True,
)

emotion_hubert_classifier_trainer = Trainer(
    model=emotion_hubert_classifier_model,
    args=emotion_hubert_classifier_training_args,
    train_dataset=emotion_train_set,
    eval_dataset=emotion_val_set,
    # compute_metrics=compute_metrics,
    # optimizers=( # need to specify optimizers explicity?
    #     torch.optim.AdamW,
    #     torch.optim.lr_scheduler.LinearLR
    # ),
)

In [ ]:
emotion_hubert_classifier_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss
1,2.025000,2.131641
2,1.921400,1.917470
3,1.884200,1.837542
4,1.787900,1.652742
5,1.690000,1.539122
6,1.449600,1.550300
7,1.276100,1.269641
8,1.088200,1.290077
9,0.861500,1.347866
10,0.740400,1.145923


TrainOutput(global_step=5760, training_loss=0.8790163649453058, metrics={'train_runtime': 1128.3563, 'train_samples_per_second': 10.21, 'train_steps_per_second': 5.105, 'total_flos': 0.0, 'train_loss': 0.8790163649453058, 'epoch': 20.0})

In [ ]:
mount_gdrive()
%cd /content/emotion_hubert_classifier_model
!zip -r checkpoint-3456.zip checkpoint-3456
!du -h checkpoint-3456.zip
!mv checkpoint-3456.zip /content/gdrive/MyDrive/2023-24/24spring/6.8620/project/models/emotion_hubert
%cd /content

Mounted at /content/gdrive
/content/emotion_hubert_classifier_model
  adding: checkpoint-3456/ (stored 0%)
  adding: checkpoint-3456/optimizer.pt (deflated 7%)
  adding: checkpoint-3456/training_args.bin (deflated 51%)
  adding: checkpoint-3456/model.safetensors (deflated 7%)
  adding: checkpoint-3456/config.json (deflated 24%)
  adding: checkpoint-3456/trainer_state.json (deflated 78%)
  adding: checkpoint-3456/rng_state.pth (deflated 25%)
  adding: checkpoint-3456/scheduler.pt (deflated 56%)
1004M	checkpoint-3456.zip
/content


## General: Transplant

In [50]:
# custom huggingface config/model definition
# https://huggingface.co/docs/transformers/en/custom_models
class TransplantHubertClassifierConfig(PretrainedConfig):
    def __init__(
            self,
            hubert_model_name = hubert_model_name,
            num_classes = 2,
            transplant_hubert_checkpoint = None,
            **kwargs,
    ):
        self.hubert_model_name = hubert_model_name
        self.num_classes = num_classes
        self.transplant_hubert_checkpoint = transplant_hubert_checkpoint
        super().__init__(**kwargs)

class TransplantHubertClassifierModel(PreTrainedModel):
    config_class = HubertClassifierConfig

    def __init__(self, config):
        super().__init__(config)
        transplant_hubert = HubertClassifierModel.from_pretrained(config.transplant_hubert_checkpoint).model.hubert
        # freeze hubert params when transplanted
        for p in transplant_hubert.parameters():
            p.requires_grad = False

        transplant_classifier = HubertClassifierModel.from_pretrained(config.transplant_hubert_checkpoint).model.classifier
        self.model = HubertClassifier(
            hubert_model_name=config.hubert_model_name,
            num_classes=config.num_classes,
            transplant_hubert=transplant_hubert,
            # transplant_classifier=transplant_classifier # don't transplant classifier, except for testing
        )

    def forward(self, input_values=None, attention_mask=None, labels=None):
        return self.model.forward(input_values=input_values, attention_mask=attention_mask, labels=labels)

## Emotion-Sarcasm Hubert

In [ ]:
# load emotion checkpoint
!gdown 1P4hapAd2-RYsbzKz0waiKpoD8bLHtdbX
!unzip checkpoint-3456.zip

Downloading...
From (original): https://drive.google.com/uc?id=1P4hapAd2-RYsbzKz0waiKpoD8bLHtdbX
From (redirected): https://drive.google.com/uc?id=1P4hapAd2-RYsbzKz0waiKpoD8bLHtdbX&confirm=t&uuid=c1f8c356-275b-4d5c-bc36-f1fbfac6dff7
To: /content/checkpoint-3456.zip
100% 1.05G/1.05G [00:25<00:00, 40.8MB/s]
Archive:  checkpoint-3456.zip
   creating: checkpoint-3456/
  inflating: checkpoint-3456/optimizer.pt  
  inflating: checkpoint-3456/training_args.bin  
  inflating: checkpoint-3456/model.safetensors  
  inflating: checkpoint-3456/config.json  
  inflating: checkpoint-3456/trainer_state.json  
  inflating: checkpoint-3456/rng_state.pth  
  inflating: checkpoint-3456/scheduler.pt  


In [ ]:
# base model testing
emotion_sarcasm_hubert_config = TransplantHubertClassifierConfig(
    num_classes=2,
    transplant_hubert_checkpoint='checkpoint-3456'
)
emotion_sarcasm_hubert = TransplantHubertClassifierModel(emotion_sarcasm_hubert_config)

# e = HubertClassifierModel.from_pretrained('checkpoint-3456')

# train_losses, val_losses = train(e, sarcasm_train_loader, sarcasm_val_loader, num_epochs=4)

Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

transplanted hubert!


In [ ]:
count_params(emotion_sarcasm_hubert)

24674

In [ ]:
new_emotion_hubert_classifier_training_args = TrainingArguments(
    output_dir='emotion_sarcasm_hubert',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=5e-5,
    weight_decay=1e-2,
    num_train_epochs=20,
    # logging_steps=100,
    lr_scheduler_type='linear',
    evaluation_strategy='epoch', # eval_strategy?
    save_strategy='epoch',
    logging_strategy='epoch',
    # load_best_model_at_end=True,
)

new_emotion_hubert_classifier_trainer = Trainer(
    model=emotion_sarcasm_hubert,
    args=new_emotion_hubert_classifier_training_args,
    train_dataset=sarcasm_train_set,
    eval_dataset=sarcasm_val_set,
    # compute_metrics=compute_metrics,
    # optimizers=( # need to specify optimizers explicity?
    #     torch.optim.AdamW,
    #     torch.optim.lr_scheduler.LinearLR
    # ),
)

In [ ]:
new_emotion_hubert_classifier_trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss
1,0.693600,0.681437
2,0.678100,0.666476
3,0.677900,0.666878
4,0.677100,0.669884
5,0.668200,0.673033
6,0.672000,0.681748
7,0.665900,0.679741
8,0.698000,0.668653
9,0.672600,0.678820
10,0.674600,0.678398


TrainOutput(global_step=5520, training_loss=0.6759941045788751, metrics={'train_runtime': 630.4362, 'train_samples_per_second': 17.512, 'train_steps_per_second': 8.756, 'total_flos': 0.0, 'train_loss': 0.6759941045788751, 'epoch': 20.0})

In [ ]:
mount_gdrive()
%cd /content/emotion_sarcasm_hubert
!zip -r checkpoint-5520.zip checkpoint-5520
!du -h checkpoint-5520.zip
!mv checkpoint-5520.zip /content/gdrive/MyDrive/2023-24/24spring/6.8620/project/models/emotion_sarcasm_hubert
%cd /content

Mounted at /content/gdrive
/content/emotion_sarcasm_hubert
  adding: checkpoint-5520/ (stored 0%)
  adding: checkpoint-5520/optimizer.pt (deflated 8%)
  adding: checkpoint-5520/training_args.bin (deflated 51%)
  adding: checkpoint-5520/model.safetensors (deflated 7%)
  adding: checkpoint-5520/config.json (deflated 31%)
  adding: checkpoint-5520/trainer_state.json (deflated 81%)
  adding: checkpoint-5520/rng_state.pth (deflated 25%)
  adding: checkpoint-5520/scheduler.pt (deflated 56%)
335M	checkpoint-5520.zip
/content


## Sarcasm-Emotion Hubert

In [51]:
# load sarcasm checkpoint
!gdown 1Em_Z2WRQY6yu7iWNSYu9xLo8NDB7FBh_
!unzip checkpoint-1104.zip

Downloading...
From (original): https://drive.google.com/uc?id=1Em_Z2WRQY6yu7iWNSYu9xLo8NDB7FBh_
From (redirected): https://drive.google.com/uc?id=1Em_Z2WRQY6yu7iWNSYu9xLo8NDB7FBh_&confirm=t&uuid=04b6cbbc-b6b0-42d7-bb9f-5111ae04788d
To: /content/checkpoint-1104.zip
100% 1.05G/1.05G [00:14<00:00, 75.1MB/s]
Archive:  checkpoint-1104.zip
   creating: checkpoint-1104/
  inflating: checkpoint-1104/optimizer.pt  
  inflating: checkpoint-1104/training_args.bin  
  inflating: checkpoint-1104/model.safetensors  
  inflating: checkpoint-1104/config.json  
  inflating: checkpoint-1104/trainer_state.json  
  inflating: checkpoint-1104/rng_state.pth  
  inflating: checkpoint-1104/scheduler.pt  


In [52]:
# base model testing
sarcasm_emotion_hubert_config = TransplantHubertClassifierConfig(
    num_classes=8,
    transplant_hubert_checkpoint='checkpoint-1104'
)
sarcasm_emotion_hubert = TransplantHubertClassifierModel(sarcasm_emotion_hubert_config)

# e = HubertClassifierModel.from_pretrained('checkpoint-3456')

# train_losses, val_losses = train(e, sarcasm_train_loader, sarcasm_val_loader, num_epochs=4)

Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

transplanted hubert!


In [53]:
sarcasm_emotion_hubert_classifier_training_args = TrainingArguments(
    output_dir='sarcasm_emotion_hubert',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=5e-5,
    weight_decay=1e-2,
    num_train_epochs=20,
    # logging_steps=100,
    lr_scheduler_type='linear',
    evaluation_strategy='epoch', # eval_strategy?
    save_strategy='epoch',
    logging_strategy='epoch',
    # load_best_model_at_end=True,
)

sarcasm_emotion_hubert_classifier_trainer = Trainer(
    model=sarcasm_emotion_hubert,
    args=sarcasm_emotion_hubert_classifier_training_args,
    train_dataset=emotion_train_set,
    eval_dataset=emotion_val_set,
    # compute_metrics=compute_metrics,
    # optimizers=( # need to specify optimizers explicity?
    #     torch.optim.AdamW,
    #     torch.optim.lr_scheduler.LinearLR
    # ),
)

In [56]:
sarcasm_emotion_hubert_classifier_trainer.train()

Epoch,Training Loss,Validation Loss
1,2.068400,2.075519
2,2.067600,2.073698
3,2.066100,2.078223
4,2.066400,2.076091
5,2.061600,2.077411
6,2.065100,2.077406
7,2.066200,2.076551
8,2.065500,2.077134
9,2.068100,2.076619
10,2.065100,2.075911


TrainOutput(global_step=5760, training_loss=2.0654622077941895, metrics={'train_runtime': 298.344, 'train_samples_per_second': 38.613, 'train_steps_per_second': 19.307, 'total_flos': 0.0, 'train_loss': 2.0654622077941895, 'epoch': 20.0})

In [58]:
mount_gdrive()
%cd /content/sarcasm_emotion_hubert
!zip -r checkpoint-5760.zip checkpoint-5760
!du -h checkpoint-5760.zip
!mv checkpoint-5760.zip /content/gdrive/MyDrive/2023-24/24spring/6.8620/project/models/sarcasm_emotion_hubert
%cd /content

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/sarcasm_emotion_hubert
  adding: checkpoint-5760/ (stored 0%)
  adding: checkpoint-5760/optimizer.pt (deflated 8%)
  adding: checkpoint-5760/training_args.bin (deflated 51%)
  adding: checkpoint-5760/model.safetensors (deflated 7%)
  adding: checkpoint-5760/config.json (deflated 31%)
  adding: checkpoint-5760/trainer_state.json (deflated 81%)
  adding: checkpoint-5760/rng_state.pth (deflated 25%)
  adding: checkpoint-5760/scheduler.pt (deflated 56%)
335M	checkpoint-5760.zip
/content


## Tutorials

### Tutorial: HubertForCTC example

https://huggingface.co/docs/transformers/en/model_doc/hubert#transformers.HubertForCTC.forward.example

In [ ]:
hubert_model_name = 'facebook/hubert-base-ls960'
processor = AutoProcessor.from_pretrained('facebook/hubert-large-ls960-ft') # is using a processor different from the model ok?
model = HubertModel.from_pretrained(hubert_model_name) # 94M params

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model ch

In [ ]:
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for hf-internal-testing/librispeech_asr_demo contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hf-internal-testing/librispeech_asr_demo
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
sampling_rate = dataset.features['audio'].sampling_rate
inputs = processor(dataset[0]['audio']['array'], sampling_rate=sampling_rate, return_tensors='pt')
with torch.no_grad():
    output = model(**inputs)
    pooler_output = torch.mean(output.last_hidden_state, dim=1) # no pooler_output provided by hubert, calculate by taking average over sequence dimension
inputs

{'input_values': tensor([[0.0386, 0.0337, 0.0322,  ..., 0.0070, 0.0095, 0.0169]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}

In [ ]:
processor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='facebook/hubert-large-ls960-ft', vocab_size=32, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	1: AddedToken("<s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	2: AddedToken("</s>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	3: AddedToken("<unk>", rstrip=True, lstrip=Tr

### Tutorial: HubertModel example

https://huggingface.co/docs/transformers/en/model_doc/hubert#transformers.HubertModel.forward.example

What is `ds['speech']`?

In [ ]:
processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
model = HubertModel.from_pretrained("facebook/hubert-base-ls960")

Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

In [ ]:
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")

In [ ]:
audio = ds[0]['audio']['array']
speech, sr = sf.read(ds[0]['file'])
all(audio == speech)

True

### Tutorial: audio classification

https://huggingface.co/docs/transformers/en/tasks/audio_classification

In [ ]:
minds_dataset = load_dataset('PolyAI/minds14', name='en-US', split='train')
minds = minds_dataset.train_test_split(test_size=0.2)
print(minds['train'])

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 450
})


In [ ]:
minds = minds.remove_columns(["path", "transcription", "english_transcription", "lang_id"])
print(minds['train'])
minds['train'][0]

Dataset({
    features: ['audio', 'intent_class'],
    num_rows: 450
})


{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/en-US~BALANCE/602b9c72bb1e6d0fbce91f87.wav',
  'array': array([ 0.00024414, -0.00024414,  0.        , ...,  0.01037598,
          0.0098877 ,  0.0098877 ]),
  'sampling_rate': 8000},
 'intent_class': 4}

In [ ]:
labels = minds['train'].features['intent_class'].names
print(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label[str(2)]

['abroad', 'address', 'app_error', 'atm_limit', 'balance', 'business_loan', 'card_issues', 'cash_deposit', 'direct_debit', 'freeze', 'high_value_payment', 'joint_account', 'latest_transactions', 'pay_bill']


'app_error'

In [ ]:
feature_extractor = AutoFeatureExtractor.from_pretrained('facebook/wav2vec2-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
minds = minds.cast_column('audio', Audio(sampling_rate=16_000))
minds['train'][0]

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/28aa727f91fee90575c34956bab09d1716cfaf460c6afcba86a10f04a7d58b83/en-US~BALANCE/602b9c72bb1e6d0fbce91f87.wav',
  'array': array([ 2.35084444e-04,  3.73781659e-05, -2.34791543e-04, ...,
          1.11566903e-02,  9.70520079e-03,  4.98228893e-03]),
  'sampling_rate': 16000},
 'intent_class': 4}

In [ ]:
def preprocess_function(examples):
    audio_arrays = [example['array'] for example in examples['audio']]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

encoded_minds = minds.map(preprocess_function, remove_columns='audio', batched=True)
encoded_minds = encoded_minds.rename_column('intent_class', 'label')
encoded_minds

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values'],
        num_rows: 450
    })
    test: Dataset({
        features: ['label', 'input_values'],
        num_rows: 113
    })
})

In [ ]:
accuracy = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(prediction=predictions, references=eval_pred.label_ids)

In [ ]:
num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    'facebook/wav2vec2-base', num_labels=num_labels, label2id=label2id, id2label=id2label
)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_mind_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss


TypeError: object of type 'NoneType' has no len()

In [ ]:
encoded_minds['test']

Dataset({
    features: ['label', 'input_values'],
    num_rows: 113
})

# Evaluation

In [ ]:
def eval(hubert_model, test_loader):
    preds_tensor = torch.tensor([]).to(DEVICE)
    labels_tensor = torch.tensor([]).to(DEVICE)

    hubert_model.eval()
    for ix, batch in tqdm.tqdm(enumerate(test_loader)):
        out = hubert_model(
            input_values=batch['input_values'].to(DEVICE),
            attention_mask=batch['attention_mask'].to(DEVICE),
            labels=batch['labels'].to(DEVICE)
        )
        preds = torch.argmax(torch.softmax(out['logits'], dim=1), dim=1)
        print(f'{preds=}')
        print(f'{preds.shape=}')
        print(f'{batch["labels"]=}')
        preds_tensor = torch.concat((preds_tensor, preds), dim=0)
        labels_tensor = torch.concat((labels_tensor, batch['labels'].to(DEVICE)), dim=0)

    print(f'{preds_tensor.shape=}')
    print(f'{labels_tensor.shape=}')


In [ ]:
e_hubert = HubertClassifierModel(HubertClassifierConfig(num_classes=8)).to(DEVICE)

eval(e_hubert, emotion_test_loader)

Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

preds=tensor([4, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([4, 4])
preds=tensor([4, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([4, 7])
preds=tensor([2, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([1, 6])


6it [00:00, 15.16it/s]

preds=tensor([4, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([0, 1])
preds=tensor([4, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([1, 5])
preds=tensor([4, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([7, 2])
preds=tensor([4, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([7, 7])


10it [00:00, 16.87it/s]

...

preds=tensor([4, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([1, 4])
preds=tensor([4, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([5, 2])
preds=tensor([4, 2], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([5, 5])
preds=tensor([4, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([1, 2])


34it [00:01, 18.14it/s]

preds=tensor([4, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([5, 3])
preds=tensor([2, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([7, 7])
preds=tensor([1, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([3, 1])
preds=tensor([4, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([7, 4])


36it [00:02, 17.29it/s]

preds=tensor([7, 4], device='cuda:0')
preds.shape=torch.Size([2])
batch["labels"]=tensor([2, 6])
preds_tensor.shape=torch.Size([72])
labels_tensor.shape=torch.Size([72])


# Visualization